In [1]:
import sys
sys.path.append(".")
import numpy as np
from Embedding import *
from base_layer import *

In [ ]:
import numpy as np

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx, np.zeros_like(Wh), np.zeros_like(b))]
        
        # 역전파 계산 시 사용하는 중간 데이터
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next
        
    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next * (1-h_next**2)
        db = np.sum(dt.sum, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(dt, Wh.T)
        dWx = np.matmul(x.T, dt)
        dx = np.matmul(dt, Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dx, dh_prev

In [ ]:
import numpy as np

class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        
        # RNN계층을 리스트로 저장할 변수
        self.layers = None
        
        self.h , self.dh = None, None
        # 이전 상태의 은닉상태를 유지할지를 지정(Truncated BPTT => False)
        self.stateful = stateful
        
    def self_state(self,h):
        self.h = h
        
    def reset_state(self):
        self.h = None
        
    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape
        
        self.layers = []
        hs = np.empty((N,T,H), dtype="f")
        
        # 첫번째 RNN 계층이거나 이전 은닉상태를 받지 않는다면 0으로 초기화
        if not self.stateful or self.h is None:
            self.h = np.zeros((N,H), dtype = "f")

        for t in range(T):
            
            # self.params에 저장되어 있는 것을 인자로 넣어줌
            layer = RNN(*self.params) 
            self.h = layer.forward(xs[:,t,:], self.h)
            hs[:,t,:] = self.h
            self.layers.append(layer)
            
        return hs
    
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape
        
        dxs = np.empty((N,T,D), dtype="f")
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t] # RNN layer 추가
            
            # 두 갈래에서 dh(t)를 받기 때문에 합
            dx, dh = layer.backward(dhs[:,t,:]+dh)
            dxs[:,t,:] = dx
            
            # 같은 가중치를 공유하기 때문에 모두 더함
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
        
        # 더한 것을 인스턴스 변수로 저장
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh
        
        return dxs

In [ ]:
class TimeEmbedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.layers = None
        self.W = W
    
    def forward(self, xs):
        # N : 데이터 개수
        N, T = xs.shape
        V, D = self.W.shape
        
        # 각 임베딩 후의 출력
        out = np.empty((N, T, D), dtype = "f")
        self.layers = []
        
        # 각 layer(time)마다 임베딩 후 출력 결과 저장
        for t in range(T):
            layer = Embedding(self.W)
            out[:,t,:] = layer.forward(xs[:,t])
            self.layers.append(layer)
        
        return out
    
    def backward(self, dout):
        N, T, D = dout.shape
        
        grad = 0
        
        # 같은 embedding matrix이기 때문에 모두 더함
        for t in range(T):
            layer = self.layers[t]
            layer.backward(dout[:, t, :])
            grad += layer.grads[0]
            
        self.grads[0][...] = grad
        

In [ ]:
class TimeAffine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
        
    def forward(self, x):
        N, T, D = x.shape
        W, b = self.params
        
        # 계산을 효율적으로 하기 위해 형태를 바꿈
        rx = x.reshape(N*T, -1)
        out = np.dot(rx, W) + b
        self.x = x
        return out.reshape(N,T,-1)
    
    def backward(self, dout):
        x = self.x
        N, T, D = x.shape
        W, b = self.params
        
        dout = dout.reshape(N*T, -1)
        rx = x.reshape(N*t, -1)
        
        db = np.sum(dout, axis=0)
        dW = np.dot(rx.T, dout)
        dx = np.dot(dout, W.T)
        dx = dx.reshape(*x.shape)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        
        return dx

In [ ]:
class TimeSoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None
        self.ignore_label = -1
    
    def forward(self, xs, ts):
        N, T, V = xs.shape
        
        if ts.ndim == 3: # 정답 레이블이 원핫 벡터인 경우
            ts = ts.argmax(axis=2) # label로 바꾸어줌
        
        mask = (ts != self.ignore_label)
        
        # 배치용과 시계열용을 정리
        xs = xs.reshape(N*T, V)
        ts = ts.reshape(N*T)
        mask = mask.reshape(N*T)
        
        ys = softmax(xs)
        ls = np.log(ys[np.arange(N*T), ts])
        ls *= mask # ignore_label에 해당하는 데이터는 손실을 0으로 설정
        loss = -np.sum(ls)
        loss /= mask.sum()
        
        self.cache = (ts,ys,mask,(N,T,V))
        return loss
    
    def backward(self, dout=1):
        ts, ys, mask, (N, T, V) = self.cache
        
        dx = ys
        dx[np.arange(N*T), ts] -=1
        dx *= dout
        dx /= mask.sum()
        dx *= mask[:, np.newaxis] # ignore_label에 해당하는 데이터는 기울기를 0으로 설정
        
        dx = dx.reshape((N, T, V))
        
        return dx